In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Embedding, Flatten, concatenate, Dropout, add
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import MultiHeadAttention, LayerNormalization, GlobalAveragePooling1D
from tensorflow.keras.optimizers import Adam

In [ ]:
from typing_extensions import final
# load the final matrix
final_matrix = pd.read_csv('final_matrix.csv')

# find the 'last' week
global_last_week = final_matrix['week_numbers'].apply(lambda x: max(eval(x))).max()

# prepare the inputs and outputs
X_weeks = final_matrix['week_numbers'].apply(eval).to_numpy()  # Haftaları numpy array'e çeviriyoruz
X_categories = final_matrix['categories'].apply(eval).to_numpy()  # Kategorileri numpy array'e çeviriyoruz
Y = final_matrix['week_numbers'].apply(
    lambda weeks: 0 if all(w <= global_last_week for w in eval(weeks)) else
    [w - global_last_week for w in eval(weeks) if global_last_week < w <= global_last_week + 4][0] if
    any(global_last_week < w <= global_last_week + 4 for w in eval(weeks)) else 0
).to_numpy()

# split the train and test sets
X_weeks_train, X_weeks_test, X_cats_train, X_cats_test, Y_train, Y_test = train_test_split(
    X_weeks, X_categories, Y, test_size=0.2, random_state=42)



In [ ]:
# padding for the categories
max_category_length = 10  # Maksimum kategori uzunluğu
X_cats_train_padded = pad_sequences(X_cats_train, maxlen=max_category_length, padding='post')
X_cats_test_padded = pad_sequences(X_cats_test, maxlen=max_category_length, padding='post')

model

In [ ]:
# inputs for the weeks
input_weeks = Input(shape=(1,), name='week_input')

# dense layer for the weeks
# apply dropout to prevent overfit
dense_weeks = Dense(256, activation='relu')(input_weeks)
dense_weeks = Dropout(0.3)(dense_weeks)
dense_weeks = Dense(64, activation='relu')(dense_weeks)
dense_weeks = Dropout(0.3)(dense_weeks)

# inputs for the categories and the embedding layer
input_cats = Input(shape=(max_category_length,), name='category_input')
embedding = Embedding(input_dim=5000, output_dim=8, input_length=max_category_length)(input_cats)
flatten_cats = Flatten()(embedding)

# concat categories and weeks
combined = concatenate([dense_weeks, flatten_cats])

# dense layers for the combined input
# apply l2 regularization to avoid overfit (ağırlıklara ceza ekleyerek)
dense_combined = Dense(16, activation='relu', kernel_regularizer=l2(0.01))(combined)
dense_combined = Dense(4, activation='relu', kernel_regularizer=l2(0.01))(dense_combined)

# output layer: 5 class (0, 1, 2, 3, 4)
output = Dense(5, activation='softmax', name='output')(dense_combined)

# define the model
model = Model(inputs=[input_weeks, input_cats], outputs=output)

# compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

model.summary()



/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ week_input (InputLayer)   │ (None, 1)              │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense (Dense)             │ (None, 256)            │            512 │ week_input[0][0]       │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dropout (Dropout)         │ (None, 256)            │              0 │ dense[0][0]            │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ category_input            │ (None, 10)             │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_1 (Dense)           │ (None, 64)             │         16,448 │ dropout[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ embedding (Embedding)     │ (None, 10, 8)          │         40,000 │ category_input[0][0]   │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dropout_1 (Dropout)       │ (None, 64)             │              0 │ dense_1[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ flatten (Flatten)         │ (None, 80)             │              0 │ embedding[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ concatenate (Concatenate) │ (None, 144)            │              0 │ dropout_1[0][0],       │
│                           │                        │                │ flatten[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_2 (Dense)           │ (None, 16)             │          2,320 │ concatenate[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_3 (Dense)           │ (None, 4)              │             68 │ dense_2[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ output (Dense)            │ (None, 5)              │             25 │ dense_3[0][0]          │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 59,373 (231.93 KB)

 Trainable params: 59,373 (231.93 KB)

 Non-trainable params: 0 (0.00 B)

train the model

In [ ]:
# if elements in X_weeks_train are not list, convert them into list
if not isinstance(X_weeks_train[0], list):
    X_weeks_train = [eval(weeks) if isinstance(weeks, str) else [weeks] for weeks in X_weeks_train]
    X_weeks_test = [eval(weeks) if isinstance(weeks, str) else [weeks] for weeks in X_weeks_test]

# convert the inputs to numpy array format
X_weeks_train = np.array([weeks[-1] if len(weeks) > 0 else 0 for weeks in X_weeks_train]).astype('float32')
X_weeks_test = np.array([weeks[-1] if len(weeks) > 0 else 0 for weeks in X_weeks_test]).astype('float32')

X_cats_train_padded = np.array(X_cats_train_padded).astype('float32')
X_cats_test_padded = np.array(X_cats_test_padded).astype('float32')

Y_train = np.array(Y_train).astype('int32')
Y_test = np.array(Y_test).astype('int32')

# output control
print("X_weeks_train shape:", X_weeks_train.shape)
print("X_cats_train_padded shape:", X_cats_train_padded.shape)
print("Y_train shape:", Y_train.shape)

# # train the model
# model.fit(
#     [X_weeks_train, X_cats_train_padded],
#     Y_train,
#     validation_data=([X_weeks_test, X_cats_test_padded], Y_test),
#     epochs=10,
#     batch_size=32
# )

# # evaluate the model through the testset
# loss, accuracy = model.evaluate([X_weeks_test, X_cats_test_padded], Y_test)
# print("Test Seti Doğruluğu:", accuracy)

X_weeks_train shape: (717140,)
X_cats_train_padded shape: (717140, 10)
Y_train shape: (717140,)


In [ ]:
# train the model
model.fit(
    [X_weeks_train, X_cats_train_padded],
    Y_train,
    validation_data=([X_weeks_test, X_cats_test_padded], Y_test),
    epochs=10,
    batch_size=32
)

# evaluate the model through the testset
loss, accuracy = model.evaluate([X_weeks_test, X_cats_test_padded], Y_test)
print("Test Seti Doğruluğu:", accuracy)

Epoch 1/10
22411/22411 ━━━━━━━━━━━━━━━━━━━━ 54s 2ms/step - accuracy: 0.9583 - loss: 0.0810 - val_accuracy: 1.0000 - val_loss: 1.2234e-10
Epoch 2/10
22411/22411 ━━━━━━━━━━━━━━━━━━━━ 82s 2ms/step - accuracy: 1.0000 - loss: 3.8016e-11 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 3/10
22411/22411 ━━━━━━━━━━━━━━━━━━━━ 81s 2ms/step - accuracy: 1.0000 - loss: 0.0000e+00 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 4/10
22411/22411 ━━━━━━━━━━━━━━━━━━━━ 56s 2ms/step - accuracy: 1.0000 - loss: 0.0000e+00 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 5/10
22411/22411 ━━━━━━━━━━━━━━━━━━━━ 82s 2ms/step - accuracy: 1.0000 - loss: 0.0000e+00 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 6/10
22411/22411 ━━━━━━━━━━━━━━━━━━━━ 56s 2ms/step - accuracy: 1.0000 - loss: 0.0000e+00 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 7/10
22411/22411 ━━━━━━━━━━━━━━━━━━━━ 51s 2ms/step - accuracy: 1.0000 - loss: 0.0000e+00 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 8/10
22411/

overfite uğradık

trying early stopping to avoid overfit

In [ ]:
early_stop = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

model.fit(
    [X_weeks_train, X_cats_train_padded],
    Y_train,
    validation_data=([X_weeks_test, X_cats_test_padded], Y_test),
    epochs=10,  # Daha fazla epoch kullanabiliriz
    batch_size=32,
    callbacks=[early_stop]
)

Epoch 1/10
22411/22411 ━━━━━━━━━━━━━━━━━━━━ 72s 3ms/step - accuracy: 0.9995 - loss: 0.0209 - val_accuracy: 1.0000 - val_loss: 1.2093e-08
Epoch 2/10
22411/22411 ━━━━━━━━━━━━━━━━━━━━ 82s 3ms/step - accuracy: 1.0000 - loss: 1.0330e-08 - val_accuracy: 1.0000 - val_loss: 9.5924e-09
Epoch 3/10
22411/22411 ━━━━━━━━━━━━━━━━━━━━ 70s 3ms/step - accuracy: 1.0000 - loss: 1.0585e-08 - val_accuracy: 1.0000 - val_loss: 1.3290e-08
Epoch 4/10
22411/22411 ━━━━━━━━━━━━━━━━━━━━ 77s 3ms/step - accuracy: 1.0000 - loss: 1.0665e-08 - val_accuracy: 1.0000 - val_loss: 1.1190e-08
Epoch 5/10
22411/22411 ━━━━━━━━━━━━━━━━━━━━ 70s 3ms/step - accuracy: 1.0000 - loss: 1.0671e-08 - val_accuracy: 1.0000 - val_loss: 1.2090e-08


test data üzerinde deneyip prediction yapıyoruz

In [ ]:
test_data = pd.read_csv('test.csv')

# prepare the data
X_cats_test_padded = pad_sequences(X_cats_test, maxlen=max_category_length, padding='post').astype('int32')
X_weeks_test = X_weeks_test.astype('float32')
X_cats_test = test_data['product_id'].apply(lambda x: [x]).values

# add padding (uzunluk eşitlemek için boşluk doldurduk)
max_category_length = 10
X_cats_test_padded = pad_sequences(X_cats_test, maxlen=max_category_length, padding='post')

In [ ]:
# prediction
predictions = model.predict([X_weeks_test, X_cats_test_padded])

predicted_classes = predictions.argmax(axis=1)
test_data['prediction'] = predicted_classes
test_data.to_csv('submission.csv', index=False)

InvalidArgumentError: Graph execution error:

Detected at node functional_1/embedding_1/GatherV2 defined at (most recent call last):
  File "/usr/lib/python3.10/runpy.py", line 196, in _run_module_as_main

  File "/usr/lib/python3.10/runpy.py", line 86, in _run_code

  File "/usr/local/lib/python3.10/dist-packages/colab_kernel_launcher.py", line 37, in <module>

  File "/usr/local/lib/python3.10/dist-packages/traitlets/config/application.py", line 992, in launch_instance

  File "/usr/local/lib/python3.10/dist-packages/ipykernel/kernelapp.py", line 619, in start

  File "/usr/local/lib/python3.10/dist-packages/tornado/platform/asyncio.py", line 195, in start

  File "/usr/lib/python3.10/asyncio/base_events.py", line 603, in run_forever

  File "/usr/lib/python3.10/asyncio/base_events.py", line 1909, in _run_once

  File "/usr/lib/python3.10/asyncio/events.py", line 80, in _run

  File "/usr/local/lib/python3.10/dist-packages/tornado/ioloop.py", line 685, in <lambda>

  File "/usr/local

BURADAN DEVAM EDİYORUZ TRANSFORMER YAPISI İLE

In [ ]:
# load the final_matrix
final_matrix = pd.read_csv('final_matrix.csv')

# find the global last week
global_last_week = final_matrix['week_numbers'].apply(lambda x: max(eval(x))).max()

# Label column
final_matrix['label'] = final_matrix['week_numbers'].apply(
    lambda weeks: 0 if all(w <= global_last_week for w in eval(weeks)) else
    [w - global_last_week for w in eval(weeks) if global_last_week < w <= global_last_week + 4][0] if
    any(global_last_week < w <= global_last_week + 4 for w in eval(weeks)) else 0
)

# necessary columns
X_customers = final_matrix['customer_id'].values
X_products = final_matrix['product_id'].apply(lambda x: [x]).values  # Her ürün bir sekans
X_weeks = final_matrix['week_numbers'].apply(eval).values  # Haftalar zaten list olarak
X_categories = final_matrix['categories'].apply(eval).values  # Kategoriler zaten list olarak
Y = final_matrix['label'].values  # Hedef değişken (0, 1, 2, 3, 4)

# sekans uzunluğu (?)
max_weeks_length = 10  # Maksimum hafta uzunluğu
max_category_length = 10  # Maksimum kategori uzunluğu

# padding
X_products_padded = pad_sequences(X_products, maxlen=max_weeks_length, padding='post')
X_weeks_padded = pad_sequences(X_weeks, maxlen=max_weeks_length, padding='post')
X_categories_padded = pad_sequences(X_categories, maxlen=max_category_length, padding='post')

# train-test splitting
X_customers_train, X_customers_test, \
X_products_train, X_products_test, \
X_weeks_train, X_weeks_test, \
X_categories_train, X_categories_test, \
Y_train, Y_test = train_test_split(
    X_customers, X_products_padded, X_weeks_padded, X_categories_padded, Y,
    test_size=0.2, random_state=42
)

create the transformer model

In [ ]:
# inputs of customer_id
input_customers = Input(shape=(1,), name='customer_input')
num_customers = X_customers.max() + 1  # Müşteri sayısına göre input_dim ayarlandı
embedding_customers = Embedding(input_dim=num_customers, output_dim=16)(input_customers)
flatten_customers = Flatten()(embedding_customers)

# product_id inputs
input_products = Input(shape=(max_weeks_length,), name='product_input')
num_products = X_products_padded.max() + 1  # Ürün sayısına göre input_dim ayarlandı
embedding_products = Embedding(input_dim=num_products, output_dim=16)(input_products)


# input layer for the categories
input_categories = Input(shape=(max_category_length,), name='category_input')
num_categories = max([max(categories) for categories in X_categories if len(categories) > 0]) + 1  # Kategorilerin sayısına göre input_dim ayarlandı
embedding_categories = Embedding(input_dim=num_categories, output_dim=8)(input_categories)
flatten_categories = Flatten()(embedding_categories)


# positional encoding (weeks)
input_weeks = Input(shape=(max_weeks_length,), name='week_input')
num_weeks = X_weeks_padded.max() + 1  # Haftaların sayısına göre input_dim ayarlandı
embedding_weeks = Embedding(input_dim=num_weeks, output_dim=16)(input_weeks)

# Combine Product ve Positional Embeddings
product_positional = tf.keras.layers.Add()([embedding_products, embedding_weeks])

# Transformer Encoder Block
attention_output = MultiHeadAttention(num_heads=4, key_dim=16)(product_positional, product_positional)
attention_output = Dropout(0.3)(attention_output)
attention_output = LayerNormalization(epsilon=1e-6)(attention_output)

# Global Average Pooling (Sekansı sabit boyutlu hale getirmek için)
pooled_output = GlobalAveragePooling1D()(attention_output)

# concatenating the customers and categories
combined = tf.keras.layers.Concatenate()([flatten_customers, pooled_output, flatten_categories])

# Dense Layers
dense_output = Dense(64, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.01))(combined)
dense_output = Dropout(0.3)(dense_output)
dense_output = Dense(16, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.01))(dense_output)
dense_output = Dropout(0.3)(dense_output)

# output layer
output = Dense(5, activation='softmax', name='output')(dense_output)



In [ ]:
# model defining and compiling
model = Model(inputs=[input_customers, input_products, input_weeks, input_categories], outputs=output)
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

model.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ product_input             │ (None, 10)             │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ week_input (InputLayer)   │ (None, 10)             │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ embedding_5 (Embedding)   │ (None, 10, 16)         │        524,416 │ product_input[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ embedding_7 (Embedding)   │ (None, 10, 16)         │            576 │ week_input[0][0]       │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ add_1 (Add)               │ (None, 10, 16)         │              0 │ embedding_5[0][0],     │
│                           │                        │                │ embedding_7[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ multi_head_attention_1    │ (None, 10, 16)         │          4,304 │ add_1[0][0],           │
│ (MultiHeadAttention)      │                        │                │ add_1[0][0]            │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ customer_input            │ (None, 1)              │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dropout_5 (Dropout)       │ (None, 10, 16)         │              0 │ multi_head_attention_… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ category_input            │ (None, 10)             │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ embedding_4 (Embedding)   │ (None, 1, 16)          │        738,208 │ customer_input[0][0]   │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ layer_normalization_1     │ (None, 10, 16)         │             32 │ dropout_5[0][0]        │
│ (LayerNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ embedding_6 (Embedding)   │ (None, 10, 8)          │         34,400 │ category_input[0][0]   │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ flatten_2 (Flatten)       │ (None, 16)             │              0 │ embedding_4[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ global_average_pooling1d… │ (None, 16)             │              0 │ layer_normalization_1… │
│ (GlobalAveragePooling1D)  │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ flatten_3 (Flatten)       │ (None, 80)             │              0 │ embedding_6[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ concatenate_1             │ (None, 112)            │              0 │ flatten_2[0][0],       │
│ (Concatenate)        

 Total params: 1,310,293 (5.00 MB)

 Trainable params: 1,310,293 (5.00 MB)

 Non-trainable params: 0 (0.00 B)

hata ayıklama kısmı BAŞLANGIÇ

In [ ]:
print("Max Customer ID:", X_customers.max())
print("Max Product ID:", X_products.max())
print("Max Category ID:", max([max(categories) for categories in X_categories if len(categories) > 0]))


Max Customer ID: 46137
Max Product ID: [32775]
Max Category ID: 4299


In [ ]:
empty_categories = [categories for categories in X_categories if len(categories) == 0]
print(f"Boş kategori sayısı: {len(empty_categories)}")


Boş kategori sayısı: 0


In [ ]:
X_categories = [categories if len(categories) > 0 else [0] for categories in X_categories]


In [ ]:
print("Max Customer ID:", X_customers.max())
print("Max Product ID:", X_products.max())
print("Max Category ID:", max([max(categories) for categories in X_categories]))


Max Customer ID: 46137
Max Product ID: [32775]
Max Category ID: 4299


In [ ]:
print(f"Max Product ID: {X_products.max()}")
print(f"Max Week Number: {X_weeks.max()}")
print(f"Max Category ID: {max([max(categories) for categories in X_categories if len(categories) > 0])}")


Max Product ID: [32775]
Max Week Number: [35, 35, 35]
Max Category ID: 4299


In [ ]:
print(f"X_customers_train shape: {X_customers_train.shape}")
print(f"X_products_train shape: {X_products_train.shape}")
print(f"X_weeks_train shape: {X_weeks_train.shape}")
print(f"X_categories_train shape: {X_categories_train.shape}")
print(f"Y_train shape: {Y_train.shape}")


X_customers_train shape: (717140,)
X_products_train shape: (717140, 10)
X_weeks_train shape: (717140, 10)
X_categories_train shape: (717140, 10)
Y_train shape: (717140,)


hata ayıklama kısmı BİTİŞ

train the model

In [ ]:
history = model.fit(
    [X_customers_train, X_products_train, X_weeks_train, X_categories_train],
    Y_train,
    validation_data=([X_customers_test, X_products_test, X_weeks_test, X_categories_test], Y_test),
    epochs=10,
    batch_size=32
)

# evaluate the model on the test set
loss, accuracy = model.evaluate([X_customers_test, X_products_test, X_weeks_test, X_categories_test], Y_test)
print("Test Seti Doğruluğu:", accuracy)

Epoch 1/10
22411/22411 ━━━━━━━━━━━━━━━━━━━━ 386s 17ms/step - accuracy: 0.9993 - loss: 0.0404 - val_accuracy: 1.0000 - val_loss: 2.1570e-06
Epoch 2/10
22411/22411 ━━━━━━━━━━━━━━━━━━━━ 381s 17ms/step - accuracy: 1.0000 - loss: 1.3827e-06 - val_accuracy: 1.0000 - val_loss: 4.3128e-08
Epoch 3/10
22411/22411 ━━━━━━━━━━━━━━━━━━━━ 381s 17ms/step - accuracy: 1.0000 - loss: 1.6128e-07 - val_accuracy: 1.0000 - val_loss: 3.1932e-08
Epoch 4/10
22411/22411 ━━━━━━━━━━━━━━━━━━━━ 377s 17ms/step - accuracy: 1.0000 - loss: 3.4832e-08 - val_accuracy: 1.0000 - val_loss: 2.0612e-08
Epoch 5/10
22411/22411 ━━━━━━━━━━━━━━━━━━━━ 369s 16ms/step - accuracy: 1.0000 - loss: 2.5928e-07 - val_accuracy: 1.0000 - val_loss: 2.5191e-08
Epoch 6/10
22411/22411 ━━━━━━━━━━━━━━━━━━━━ 377s 16ms/step - accuracy: 1.0000 - loss: 1.6390e-07 - val_accuracy: 1.0000 - val_loss: 2.6042e-08
Epoch 7/10
22411/22411 ━━━━━━━━━━━━━━━━━━━━ 392s 17ms/step - accuracy: 1.0000 - loss: 4.7712e-08 - val_accuracy: 1.0000 - val_loss: 2.9818e-08
Epo

tahmin yap, çıktıyı kaydet